<a href="https://colab.research.google.com/github/ykim71/georgia_analysis/blob/Christian-comments/(2)_Georgia_place_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# need access to the folder(s) below:

#%cd /content/drive/MyDrive/Georgia/
#%cd /content/drive/MyDrive/Colab Notebooks/'Check code for others'/
%cd /content/drive/MyDrive/Colab Notebooks/'Check code for others'/'Check Georgia code for Yujin, apr 2023'/Georgia/
#%cd /content/drive/MyDrive/Colab Notebooks/'Check code for others'/'Check Georgia code for Yujin, apr 2023'/Georgia/'Locations Analysis'
!ls

/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia
 2020-count-by-county-population--with-2010.pdf
'Georgia Left to Do Feb 2023.gdoc'
'Georgia Media Landscape Report Draft.gdoc'
'Georgia MEETING NOTES.gdoc'
'Locations Analysis'
'Organization Analysis'
'[SHARED] GNC Research Component.gdoc'
'Topic Dictionaries'
'Web Scraping'


# Location analysis

## Prep for place analysis

In [15]:
#%cd /content/drive/MyDrive/Georgia/Locations Analysis
%cd /content/drive/MyDrive/Colab Notebooks/'Check code for others'/'Check Georgia code for Yujin, apr 2023'/Georgia/'Web Scraping' 
#%cd /content/drive/MyDrive/Colab Notebooks/'Check code for others'/'Check Georgia code for Yujin, apr 2023'/Georgia/ 
#!ls

!pwd
!ls

/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Web Scraping
/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Web Scraping
 CrowdTangle_GA.xlsx
'Georgia Batches Justification for Scraping.gdoc'
 Georgia_data_v2_translation_ner3_lem.pkl
 Georgia_data_v2_translation_ner_CSBO.pkl
'Georgia Media RFP.gdoc'
'Georgia Scraping Methods - Emily.gdoc'
'Keeping track of media sites'
'List of Potential Web Scrapers (Georgia).gdoc'
'Marquee data'
'Updated Marquee data'


In [16]:
import pandas as pd

#df = pd.read_pickle('/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl') #CO: this threw an error
df = pd.read_pickle('Georgia_data_v2_translation_ner3_lem.pkl') #CO: I assume this is the relevant file!

In [17]:
# CO: save just the first 1000 rows to make things run faster as I check the script
df = df.head(1000)


In [18]:
# lemmatization if needed
import spacy
from spacy import displacy

sp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
# CH: not familiar with "en_core_web_sm" but sounds reasonable. 
help(spacy.load)

In [20]:
def text_ner_place(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
    if entity.label_ == "FAC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "GPE":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "LOC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    else:
      None
  return str(set(ner_list))


In [21]:
df['ner_places'] = df['article_final'].astype(str).apply(text_ner_place)

In [34]:
print(df)
#CO: rows 995-999 have something weird-looking under the column "ner_person": "A1\nWord (PERSON). Not sure if that's a problem.

     index             source  en_source                            website  \
0        0  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
1        1  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
2        2  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
3        3  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
4        4  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
..     ...                ...        ...                                ...   
995    995    Clayton Tribune        1.0  http://www.theclaytontribune.com/   
996    996    Clayton Tribune        1.0  http://www.theclaytontribune.com/   
997    997    Clayton Tribune        1.0  http://www.theclaytontribune.com/   
998    998    Clayton Tribune        1.0  http://www.theclaytontribune.com/   
999    999    Clayton Tribune        1.0  http://www.theclaytontribune.com/   

                                           article_

In [22]:
df.to_pickle('Georgia_data_v2_translation_ner_CSBO.pkl') # CO: saving this but as a new file, just for myself, so I can re-load from here, when Colab throws me out 

## Find rough location from spaCy

In [23]:
import pandas as pd

df = pd.read_pickle('Georgia_data_v2_translation_ner_CSBO.pkl')


In [35]:
df_ner = df.loc[df['ner_places'] != 'set()'] #CO: so your selecting just the rows where there's something in the ner_places column, i.e., articles where some place was found. Makes sense.
print(df_ner)

     index             source  en_source                            website  \
0        0  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
1        1  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
2        2  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
3        3  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
4        4  Metter Advertiser        1.0   http://www.metteradvertiser.com/   
..     ...                ...        ...                                ...   
995    995    Clayton Tribune        1.0  http://www.theclaytontribune.com/   
996    996    Clayton Tribune        1.0  http://www.theclaytontribune.com/   
997    997    Clayton Tribune        1.0  http://www.theclaytontribune.com/   
998    998    Clayton Tribune        1.0  http://www.theclaytontribune.com/   
999    999    Clayton Tribune        1.0  http://www.theclaytontribune.com/   

                                           article_

In [25]:
print(len(df))
print(len(df_ner))

1000
777


In [26]:
df_expand = df_ner['ner_places'].str.split(',').explode().str.strip('{} ')


In [36]:
df_expand = pd.DataFrame(df_expand)
print(df_expand) #CO: in the top rows, under ner_places_clean, I see things like "Jones" and "jimmy". They're noted as places here, but sound more like people-names to me.


                    ner_places    ner_places_clean
0              'Georgia (GPE)'            Georgia 
0              'America (GPE)'            America 
1               'Metter (GPE)'             Metter 
2                'Jones (GPE)'              Jones 
2                'Jimmy (GPE)'              Jimmy 
..                         ...                 ...
998            'Clayton (GPE)'            Clayton 
998  'South Main Street (LOC)'  South Main Street 
999      'Barnyard Lane (FAC)'      Barnyard Lane 
999        'Blairsville (GPE)'        Blairsville 
999            'Clayton (GPE)'            Clayton 

[5338 rows x 2 columns]


In [28]:
df_expand['ner_places_clean'] = df_expand['ner_places'].str.replace(r"\'","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r" \"","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\"","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\(.*\)","")

<ipython-input-28-03789092516e>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_expand['ner_places_clean'] = df_expand['ner_places'].str.replace(r"\'","")
<ipython-input-28-03789092516e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r" \"","")
<ipython-input-28-03789092516e>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\"","")
<ipython-input-28-03789092516e>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\(.*\)","")


In [37]:
df_expand_count = df_expand.groupby('ner_places_clean').count().reset_index()
df_expand_count #CO: some of the items in "ner_places_clean" don't look like places. Some examples: include rows 1 (/LL), 2233 ('s Instagram), 2235 (Torch Song), and 2237 (Nausea).

,ner_places_clean,ner_places
0,-Germany Road at,1
1,/LL,2
2,102nd,1
3,110 Airport Road,1
4,120 Main Street,2
...,...,...
2233,’s Instagram,1
2234,“Mochi Land”,1
2235,“Torch Song,1
2236,•Candler County,1


In [30]:
df_freq_n5 = df_expand_count[df_expand_count["ner_places"] >= 5]
df_freq_n5.to_excel('df_freq_n5_ner_v2.xlsx')
#CO: consider doing a count which might provide relevant info for a methods section: How may articles are being filtered out (a) when subsetting to only include articles where a place was found, and (b) when filtering by NER > 5?

In [31]:
# # compare place entities with v1

# df_freq_n5_v2 = pd.read_excel('df_freq_n5_ner_v2.xlsx')
# df_freq_n5_v1 = pd.read_excel('df_freq_n5_ner.xlsx')

# df_freq_n5_v1['version'] = 'v1'
# df_freq_n5_v2['version'] = 'v2'

# df_freq_n5_merge = pd.merge( df_freq_n5_v1,df_freq_n5_v2, how="outer", on=["ner_places_clean"])

## Find sentence example of location

In [39]:
!pwd
%cd /content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia

/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Web Scraping
/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia


In [32]:
import pandas as pd #CO: already imported.

#ner_places = pd.read_csv('Georgia_ner_places.csv') # previous data
ner_places = pd.read_csv('Locations Analysis/Georgia_ner_places_v2.csv') # updated data #CO: Not seeing this file in the folder. I'm seeing an Excil file, though. Not sure if they're the same?

#CO: without this file, I can't run the code below, but I'll still check it! If you'd like me to actually run this code, let's check in and talk about which file I should use in this step.


FileNotFoundError: ignored

In [40]:
ner_places['ner_places_clean'] = ner_places['ner_places_clean'].str.rstrip()

NameError: ignored

In [ ]:
ner_places_raw = list(set(ner_places['ner_places_clean'].tolist())) #CO: small thing: Why is this object called "raw" when it's based on the clean version?

In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# df["id"] = df.index + 1
df_ner = df.loc[df['ner_places'] != 'set()'] #CO: This step has already been done once above (in chunk 21)

In [ ]:

import numpy as np

df_ner['place_ng1'] = df_ner['article_final'].astype(str).apply(lambda x: [i for i in ner_places_raw if i in x]) #CO: I'm not too familiar with this approach, and am not sure whether it's correct, but it probably is! Just to be sure, I recommend printing the created object and doucle check that it looks as you want. (I'm missing a file, so I can't print the object myself)


In [ ]:
text_df_uni_dict = df_ner[['id',  'place_ng1']]
text_df_uni_article = df_ner[['id', "article_final"]]


In [ ]:
#CO: not sure why we're taking a sample sentence here. Can't tell based on this code how this is a sample.
sample_sentence = text_df_uni_dict['place_ng1'].astype(str).str.split(',').explode().str.strip('[] ')
sample_sentence_df = pd.DataFrame(sample_sentence)

In [ ]:

merge_uni = pd.merge(sample_sentence_df, text_df_uni_article, left_index=True, right_index=True)

merge_uni['place_ng1'] = merge_uni['place_ng1'].str.strip('\'')
#CO: recommend printing merge_uni to check that it looks as you expect. Same for the objects created in the next few chunks.

In [ ]:
merge_uni_count = merge_uni.groupby('place_ng1').count() #


In [ ]:
# articles mentioned a location less than 3
merge_uni_count_l3 = merge_uni_count[merge_uni_count['article_final'] < 3].reset_index() #CO: The threshold about was 5. Why is it 3 here? Should it be consistent?


In [ ]:
merge_uni_count_l3_places = merge_uni_count_l3.place_ng1.values.tolist()
merge_uni_count_l3_places

In [ ]:
merge_uni_l3 = merge_uni[merge_uni['place_ng1'].isin(merge_uni_count_l3_places)]
merge_uni_l3_group = merge_uni_l3.groupby('place_ng1').apply(lambda x: x.sample(3, replace=True)).reset_index(drop=True)

In [ ]:
# articles mentioned a location more than 3 --> random sample

merge_uni_m3 = merge_uni[~merge_uni['place_ng1'].isin(merge_uni_count_l3_places)]
merge_uni_m3_group = merge_uni_m3.groupby('place_ng1').apply(lambda x: x.sample(3, replace=False)).reset_index(drop=True)
# CO: why are we taking a small random sample here?


In [ ]:
# merge 3 samples + those with less than 3 
merge_uni_m3_group2 = pd.concat([merge_uni_m3_group, merge_uni_l3_group]).reset_index()

In [ ]:
merge_uni_m3_group2.rename(columns={'place_ng1':'places'}, inplace=True)


In [ ]:
merge_uni_m3_group2

In [ ]:
# split an article into sentences

from nltk.tokenize import sent_tokenize
by_sentence = merge_uni_m3_group2['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)


In [ ]:
by_sentence_df

,sentence_each,sentence_turn
0,Panel recommends anxiety screening for adults ...,0
0,The proposed measure intends to identify early...,1
0,"""""Screening all adults for depression, includi...",2
0,The panel includes generalized anxiety disorde...,3
0,"According to the panel, screening and follow-u...",4
...,...,...
8288,The cause of the fire is being investigated by...,12
8288,“It’s cooling off now.,13
8288,We’re going to go in and try to collect eviden...,14
8288,Editor’s Note: This is developing with updates...,15


In [ ]:
# merging split sentences to the article level
merge_all_m3_join = pd.concat([merge_uni_m3_group2, by_sentence_df], axis=1)

#CO: Consider printing more of these objects as you go along to double check that they are as expected.

In [ ]:
merge_all_m3_join['search_words'] = merge_all_m3_join.apply(lambda x: x.places in x.sentence_each, axis=1)


In [ ]:
merge_all_m3_join['article_id'] = merge_all_m3_join.index.get_level_values(0)
merge_all_m3_join.reset_index(drop=True, inplace=True)


In [ ]:
# find those sentences that contain searching words (locations)

merge_all_m3_join_TRUE = merge_all_m3_join[merge_all_m3_join['search_words']==True]

In [ ]:
merge_temp = merge_all_m3_join_TRUE.groupby('places').count() #

#CO: Why do we have a first row with the number "22" in each col? All the cols appear identical. Is that as expected?

,index,id,article_final,sentence_each,sentence_turn,search_words,article_id
places,,,,,,,
,22,22,22,22,22,22,22
14 W. Main St.,3,3,3,3,3,3,3
1495 Roswell Road,3,3,3,3,3,3,3
169/28,3,3,3,3,3,3,3
1:48,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...
uk,5,5,5,5,5,5,5
west Lincoln,3,3,3,3,3,3,3
west Tennessee,3,3,3,3,3,3,3


In [ ]:
# extract 3 random sentences containing the searching word (location) only

merge_temp_m3_group = merge_all_m3_join_TRUE.groupby('places').apply(lambda x: x.sample(3, replace=False)).reset_index(drop=True)

#CO: not sure why we're taking 3 random sentences here?

In [ ]:
merge_temp_m3_group

,index,places,id,article_final,sentence_each,sentence_turn,search_words,article_id
0,0,,158151,Panel recommends anxiety screening for adults ...,"According to the panel, screening and follow-u...",4,True,0
1,0,,158151,Panel recommends anxiety screening for adults ...,"In 2020, the National Alliance on Mental Illne...",10,True,0
2,0,,158151,Panel recommends anxiety screening for adults ...,"""""After diagnosis, people should participate i...",14,True,0
3,3,14 W. Main St.,47531,"GRAFTON — A new attraction described as a """"th...",The coaster ride is expected to be open year-r...,10,True,3
4,4,14 W. Main St.,30084,Gazebo Downtown Franklin decorated for Fall. p...,American Legion Macon Post 108 and American Le...,13,True,4
...,...,...,...,...,...,...,...,...
8254,8271,west entrance,48058,"UNI-Dome Render 1\n\nUpdated Oct 11, 2022\n\nA...","UNI-Dome Render 6\n\nUpdated Oct 11, 2022\n\nT...",8,True,8271
8255,8272,west entrance,47270,Iowa pork producers would suffer huge negative...,"UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA...",36,True,8272
8256,8275,west.,92690,PHILADELPHIA — A local Republican Party in Sou...,"Now Richard Ringer, a former journalist, is ru...",54,True,8275
8257,8276,west.,105640,Darrell Brooks trial: Waukesha parade's younge...,He said he was concerned with 25 frames as the...,169,True,8276


In [ ]:
merge_temp_m3_group2 = merge_temp_m3_group[['places','id','sentence_each']]

In [ ]:
merge_temp_m3_group2['domainId'] = merge_temp_m3_group2.groupby('places')['places'].rank(method='first')


<ipython-input-114-32e3fd288afa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_temp_m3_group2['domainId'] = merge_temp_m3_group2.groupby('places')['places'].rank(method='first')


In [ ]:
merge_temp_m3_group2.sort_values('places')

,places,id,sentence_each,domainId
0,,158151,"According to the panel, screening and follow-u...",1.0
1,,158151,"In 2020, the National Alliance on Mental Illne...",2.0
2,,158151,"""""After diagnosis, people should participate i...",3.0
3,14 W. Main St.,47531,The coaster ride is expected to be open year-r...,1.0
4,14 W. Main St.,30084,American Legion Macon Post 108 and American Le...,2.0
...,...,...,...,...
8253,west entrance,48058,"UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA...",1.0
8255,west entrance,47270,"UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA...",3.0
8257,west.,105640,He said he was concerned with 25 frames as the...,2.0
8256,west.,92690,"Now Richard Ringer, a former journalist, is ru...",1.0


In [ ]:
merge_temp_m3_group2_pivot = merge_temp_m3_group2.pivot(index='places', columns = 'domainId', values='sentence_each').reset_index() #CO: is it correct to use "places" as the index here? Also, I wonder if any issues can be caused by using reset_index so often?


In [ ]:
merge_temp_m3_group2_pivot.rename(columns={1:'sent_1', 2:'sent_2', 3:'sent_3'}, inplace=True)
merge_temp_m3_group2_pivot #CO why three sentences? Because we took three random sentences above? But, still, why that number? Also, does each row correspond to one article? Should it?


domainId,places,sent_1,sent_2,sent_3
0,,"According to the panel, screening and follow-u...","In 2020, the National Alliance on Mental Illne...","""""After diagnosis, people should participate i..."
1,14 W. Main St.,The coaster ride is expected to be open year-r...,American Legion Macon Post 108 and American Le...,The coaster ride is expected to be open year-r...
2,1495 Roswell Road,"According to investigators, around 9:15 p.m. o...","According to police, around 9:15 p.m. Saturday...",Marietta PD’s Selective Traffic Enforcement Pr...
3,169/28,"Box: Troy Buchanan 44, Timberland 12\n\nUpdate...",1 2 3 4 Final\nVirden North Mac 15 7 14 14 50\...,"Box: Troy Buchanan 44, Timberland 12\n\nUpdate..."
4,1:48,Biden creates national monument in Colorado - ...,South Korea’s Joint Chiefs of Staff said in a ...,South Korea’s Joint Chiefs of Staff said in a ...
...,...,...,...,...
2748,uk,"The trio is one shot ahead of Ken Duke, who po...",“The DPRK clearly deems nuclear tests essentia...,"Senate Minority Leader Mitch McConnell, R-Ky.,..."
2749,west Lincoln,Police find 10 spent shell casings after gunsh...,Police find 10 spent shell casings after gunsh...,Police find 10 spent shell casings after gunsh...
2750,west Tennessee,A west Tennessee woman's accused of murdering ...,A west Tennessee woman's accused of murdering ...,A west Tennessee woman's accused of murdering ...
2751,west entrance,"UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA...","UNI-Dome Render 6\n\nUpdated Oct 11, 2022\n\nT...","UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA..."


In [ ]:
ner_places['places'] = ner_places['ner_places_clean']

In [ ]:
merge_temp_final = pd.merge(ner_places, merge_temp_m3_group2_pivot, on="places", how='left').reset_index()
#CO: recommend printing this. Does it look as expected?

In [ ]:
merge_temp_final['sent_1'] = merge_temp_final['sent_1'].str.replace('\n', ' ')
merge_temp_final['sent_2'] = merge_temp_final['sent_2'].str.replace('\n', ' ')
merge_temp_final['sent_3'] = merge_temp_final['sent_3'].str.replace('\n', ' ')

In [ ]:
# Mt. Pisgah
merge_temp_final.to_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Georgia_ner_places_v2_wsent.xlsx') #CO: I'm not seeing this file in the folders I have access to. Is it the same as "Georgia_ner_places_v2.xlsx"? Double check that the correct file is being used later?

## Look up location (w updated data) on articles

In [42]:
#%cd /content/drive/MyDrive/Georgia/Locations Analysis
%cd /content/drive/MyDrive/Colab Notebooks/'Check code for others'/'Check Georgia code for Yujin, apr 2023'/Georgia/'Locations Analysis'
!pwd


/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Locations Analysis
/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Locations Analysis


In [43]:
# data created by manual coding
#xxxxx
import pandas as pd

location_final = pd.read_excel('Georgia_place_validation_counties.xlsx') #CO: I'm not sure which part here was done manually?

In [45]:
location_final = location_final[location_final['final_selection']==1]

In [46]:
location_final_st = location_final[['ner_place_clean','county_level', 'final_selection']]

In [47]:
location_final_st = location_final_st.dropna()

In [48]:
len(location_final) #CO: include this, so we can tell how many are being dropped
len(location_final_st)

1930

In [49]:
location_final_st

,ner_place_clean,county_level,final_selection
0,110 Airport Road,Habersham County,1
1,1495 Roswell Road,Cobb County,1
2,26th Street,Fulton County,1
3,3227 Roswell Rd NE,Fulton County,1
4,46 Plaza Way,Rabun County,1
...,...,...,...
1950,Yonah Memorial Gardens,Habersham County,1
1951,York,Rabun County,1
1952,Youngstown,Union County,1
1953,Ypsilanti,Talbot County,1


In [ ]:
# # multiple counties deleted
# location_final_st = location_final_st[location_final_st['county_level'].str.contains('Counties')==False]

In [50]:
len(set(location_final_st.county_level.values.tolist()))

159

In [51]:
location_final_list = list(set(location_final_st['ner_place_clean'].tolist()))

In [89]:
#import pandas as pd #CO: already imported. Consider only importing each package once. Such a minor thing, I know, but I hope these comments are helpful. Getting rid of some clutter might make it faster to read/edit the script later on.

#df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')

%cd /content/drive/MyDrive/Colab Notebooks/'Check code for others'/'Check Georgia code for Yujin, apr 2023'/Georgia/'Web Scraping'
#!pwd
df = pd.read_pickle('Georgia_data_v2_translation_ner3_lem.pkl') #CO: I assume this is the relevant file!


/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Web Scraping


In [55]:

import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ]
  
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]

word_bigrams_done = list(set(bigrams(location_final_list)))
print(word_bigrams_done) #CO: some of these don't really seem to be bigrams. For example: 'Mill Creek High School Football Game'
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(location_final_list)
print(word_unigrams_done)
print(len(word_unigrams_done)) #CO: why are there more bigrams than unigrams? Conceptually, wouldn't we expect at least as mnay unigrams? I'm probably just missing something, but I'll mention it in case it's helpful to take a second look at!


['Presbyterian Village', 'the Washington W. King/College Avenue Bridge', 'Collins Street', 'La Vista', 'Mt. Pisgah', 'Calhoun City Schools', 'the Oconee River', 'Fort Valley State', 'the Cobb Galleria Centre', 'Red Bud', 'North Clayton', 'the 405 Freeway', 'the Rossville Public Library', 'Decatur Square', 'Winkenhofer Pine Ridge Funeral Home', 'Henderson Park', 'Butler Avenue', 'Amberly Drive', 'Cleveland Avenue', 'Valdosta State Prison', 'Inman Park', 'Bobo Street', 'Trammell Road', 'Mineral Springs Road', 'Sky Valley Way', 'Mill Creek High School Football Game', 'Goshen Road', 'Brownlee Mountain Road', 'Henry County’s', 'Pleasant Grove Road', 'Mt. Sinai Road', 'Habersham Central 1977', 'the Carson McLane Funeral Home', 'Historic District', 'Swift Cantrell Park', 'Fort Gillem', 'Hammond Park', 'Piedmont Park', 'Cedar Valley', 'Duncan Bridge Rd', 'Thomasville Heights', 'the City of Elberton', 'Bethlehem Road', 'Stephens County High School', 'Heritage Park', 'the Georgia State Prison', 

In [90]:
import numpy as np #CO: already imported above
df = df.head(5000) #CO: take a small sample to speed it up as I'm checking in
df['place_ng1'] = df['article_final'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['place_ng2'] = df['article_final'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [91]:
#df['place_ng1'] #looks like unigrams. Good.
print(df['place_ng2'].head(50)) #bigrams look good, too, although some technically don't seem like bigrams (e.g., "Kennedy Funeral Homes"). Also, I think that "Kennedy Funeral Homes" is a company, not a place.

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4                                                    []
5                                                    []
6                                                    []
7                                                    []
8                                                    []
9                                                    []
10                                                   []
11                                                   []
12                                                   []
13                                                   []
14                                                   []
15                                                   []
16                                                   []
17                                              

In [92]:
df_uni = df[(df['place_ng1'].str.len()) >= 1]
df_bi = df[(df['place_ng2'].str.len()) >= 1]

df_uni = df_uni[['id', 'source', 'place_ng1']]
df_bi = df_bi[['id', 'source', 'place_ng2']]

df_uni_df = df_uni.explode('place_ng1')
df_bi_df = df_bi.explode('place_ng2')

df_uni_df = df_uni_df.rename(columns={'place_ng1': 'place_ner'})
df_bi_df = df_bi_df.rename(columns={'place_ng2': 'place_ner'})


In [93]:
df_uni_df #CO: most these seem to be names of people (Allen, Charles, Rebecca, Philips, etc.), not places. 
df_bi_df #CO: these generally look more relevant than the unigrams

,id,source,place_ner
18,19,Metter Advertiser,East Georgia
18,19,Metter Advertiser,Georgia State
23,24,Metter Advertiser,Twin City
25,26,Metter Advertiser,Kennedy Funeral Homes
25,26,Metter Advertiser,Bulloch Memorial Gardens
...,...,...,...
4826,4827,WRDW News CBS 12 NBC 26,Mount Carmel
4877,4878,WRDW News CBS 12 NBC 26,Poplar Street
4882,4883,WRDW News CBS 12 NBC 26,Henry Street
4919,4920,WRDW News CBS 12 NBC 26,Main S


In [94]:
df_ng_all = pd.concat([df_uni_df, df_bi_df])
#df_ng_all
print(df_ng_all.head(50)) #CO: a lot of these still seem like the names of people

    id             source   place_ner
2    3  Metter Advertiser       Allen
2    3  Metter Advertiser     Charles
2    3  Metter Advertiser    Franklin
2    3  Metter Advertiser      Murphy
2    3  Metter Advertiser     Rebecca
2    3  Metter Advertiser    Kathleen
2    3  Metter Advertiser      Bailey
2    3  Metter Advertiser    Pineland
2    3  Metter Advertiser     William
2    3  Metter Advertiser     Beverly
2    3  Metter Advertiser     Roberta
2    3  Metter Advertiser       Billy
2    3  Metter Advertiser      Metter
2    3  Metter Advertiser     Randall
2    3  Metter Advertiser        Anna
3    4  Metter Advertiser      Metter
5    6  Metter Advertiser     Claxton
5    6  Metter Advertiser      Metter
5    6  Metter Advertiser       Moore
6    7  Metter Advertiser    Rockmart
6    7  Metter Advertiser      Metter
7    8  Metter Advertiser        Jake
7    8  Metter Advertiser      Bailey
7    8  Metter Advertiser      Metter
7    8  Metter Advertiser      Butler
8    9  Mett

In [95]:
count = df_ng_all.groupby(['source']).count() 
count #CO: why is the id column identical to the place_ner column?

,id,place_ner
source,,
Atlanta Business Chronicle,87,87
Banks County News,107,107
Capitol Beat,138,138
Carroll Star News,32,32
Clayton Tribune,690,690
Georgia Recorder,138,138
Metter Advertiser,162,162
Moultrie Observer,893,893
Pickens County Progress,98,98


In [97]:
count_20 = count[count['place_ner'] >= 20].reset_index()
count_20
#CO: What is the purpose? Why 20?

,source,id,place_ner
0,Atlanta Business Chronicle,87,87
1,Banks County News,107,107
2,Capitol Beat,138,138
3,Carroll Star News,32,32
4,Clayton Tribune,690,690
5,Georgia Recorder,138,138
6,Metter Advertiser,162,162
7,Moultrie Observer,893,893
8,Pickens County Progress,98,98
9,The Daily Tribune News,545,545


In [99]:
count_20_id = list(set(count_20['source'].tolist()))

df_ng_all_m20 = df_ng_all[df_ng_all['source'].isin(count_20_id)]
df_ng_all_m20 #CO: Is the purpose to filter out/drop all stories from outlets with < "place_ner" counts? If so, why do this at the outlet level, and not only at the story level?

,id,source,place_ner
2,3,Metter Advertiser,Allen
2,3,Metter Advertiser,Charles
2,3,Metter Advertiser,Franklin
2,3,Metter Advertiser,Murphy
2,3,Metter Advertiser,Rebecca
...,...,...,...
4826,4827,WRDW News CBS 12 NBC 26,Mount Carmel
4877,4878,WRDW News CBS 12 NBC 26,Poplar Street
4882,4883,WRDW News CBS 12 NBC 26,Henry Street
4919,4920,WRDW News CBS 12 NBC 26,Main S


In [100]:
len(count_20_id)

13

In [102]:
location_final_st = location_final_st.rename(columns={'ner_place_clean': 'place_ner'})
location_final_st #CO: should the two columns correspond to each other? If so, how do we know that they do? Is this what was manually validated earlier on?


,place_ner,county_level,final_selection
0,110 Airport Road,Habersham County,1
1,1495 Roswell Road,Cobb County,1
2,26th Street,Fulton County,1
3,3227 Roswell Rd NE,Fulton County,1
4,46 Plaza Way,Rabun County,1
...,...,...,...
1950,Yonah Memorial Gardens,Habersham County,1
1951,York,Rabun County,1
1952,Youngstown,Union County,1
1953,Ypsilanti,Talbot County,1


In [121]:
df['article_final'][0]

'The American Pickers are excited to return to Georgia. They plan to film episodes of The History Channel hit television series throughout the area in December 2022.\nAmerican Pickers is a documentary series that explores the fascinating world of antique “picking” on The History Channel. The hit show follows skilled pickers in the business as they hunt for America’s most valuable antiques.\nThey are always excited to find sizable, unique collections and learn the interesting stories behind them.\nAs they hit the back roads from coast to coast, the Pickers are on a mission to recycle and rescue forgotten relics.\nAlong the way, they want to meet characters with remarkable and exceptional items. They hope to give historically significant objects a new lease on life while learning a thing or two about America’s past along the way. The Pickers have seen a lot of rusty gold over the years and are always looking to discover something they’ve never seen before. They are ready to find extraord

In [117]:
print(df[['article_final'][0]][0]) #CO: print the text of the very first article, as a sanity check

The American Pickers are excited to return to Georgia. They plan to film episodes of The History Channel hit television series throughout the area in December 2022.
American Pickers is a documentary series that explores the fascinating world of antique “picking” on The History Channel. The hit show follows skilled pickers in the business as they hunt for America’s most valuable antiques.
They are always excited to find sizable, unique collections and learn the interesting stories behind them.
As they hit the back roads from coast to coast, the Pickers are on a mission to recycle and rescue forgotten relics.
Along the way, they want to meet characters with remarkable and exceptional items. They hope to give historically significant objects a new lease on life while learning a thing or two about America’s past along the way. The Pickers have seen a lot of rusty gold over the years and are always looking to discover something they’ve never seen before. They are ready to find extraordinary

In [103]:
df_ng_all_m20_county = pd.merge(df_ng_all_m20, location_final_st, how='left', on='place_ner')
df_ng_all_m20_county

,id,source,place_ner,county_level,final_selection
0,3,Metter Advertiser,Allen,Oconee County,1
1,3,Metter Advertiser,Charles,Toombs County,1
2,3,Metter Advertiser,Franklin,Heard County,1
3,3,Metter Advertiser,Murphy,Colquitt County,1
4,3,Metter Advertiser,Rebecca,Turner County,1
...,...,...,...,...,...
5404,4827,WRDW News CBS 12 NBC 26,Mount Carmel,Walker County,1
5405,4878,WRDW News CBS 12 NBC 26,Poplar Street,Bibb County,1
5406,4883,WRDW News CBS 12 NBC 26,Henry Street,Richmond County,1
5407,4920,WRDW News CBS 12 NBC 26,Main S,Gwinnett County,1


In [ ]:
df_ng_all_m20_county

,id,source,place_ner,county_level,final_selection
0,3,Metter Advertiser,Charles,Toombs County,1
1,3,Metter Advertiser,Franklin,Heard County,1
2,3,Metter Advertiser,Metter,Candler County,1
3,3,Metter Advertiser,William,Chatham County,1
4,3,Metter Advertiser,Murphy,Colquitt County,1
...,...,...,...,...,...
194285,191910,WALB TV 10 (NBC),Osprey Cove Drive,Muscogee County,1
194286,191944,WALB TV 10 (NBC),Magnolia Drive North,Effingham County,1
194287,191955,WALB TV 10 (NBC),Pine Street,Fulton County,1
194288,191985,WALB TV 10 (NBC),Wake County,Walker County,1


In [122]:
df_ng_all_m20_county_unique = df_ng_all_m20_county.drop_duplicates(subset=['id', 'source', 'county_level']) #CO: why drop duplicate values here? 

In [123]:
df_ng_all_m20_county_unique

,id,source,place_ner,county_level,final_selection
0,3,Metter Advertiser,Allen,Oconee County,1
1,3,Metter Advertiser,Charles,Toombs County,1
2,3,Metter Advertiser,Franklin,Heard County,1
3,3,Metter Advertiser,Murphy,Colquitt County,1
4,3,Metter Advertiser,Rebecca,Turner County,1
...,...,...,...,...,...
5403,4786,WRDW News CBS 12 NBC 26,Ponderosa Drive,Lowndes County,1
5404,4827,WRDW News CBS 12 NBC 26,Mount Carmel,Walker County,1
5405,4878,WRDW News CBS 12 NBC 26,Poplar Street,Bibb County,1
5406,4883,WRDW News CBS 12 NBC 26,Henry Street,Richmond County,1


In [125]:
df_ng_all_m20_county_unique_count = df_ng_all_m20_county_unique.groupby(['source','county_level'])['id'].count().reset_index()
df_ng_all_m20_county_unique_count

,source,county_level,id
0,Atlanta Business Chronicle,Bibb County,3
1,Atlanta Business Chronicle,Butts County,2
2,Atlanta Business Chronicle,Calhoun County,2
3,Atlanta Business Chronicle,Carroll County,1
4,Atlanta Business Chronicle,Cobb County,5
...,...,...,...
787,WRDW News CBS 12 NBC 26,Whitfield County,1
788,WRDW News CBS 12 NBC 26,Wilcox County,10
789,WRDW News CBS 12 NBC 26,Wilkes County,6
790,WRDW News CBS 12 NBC 26,Wilkinson County,23


In [126]:
df_ng_all_m20_county_unique_count = df_ng_all_m20_county_unique_count.rename(columns={'id': 'article_id_count'})
df_ng_all_m20_county_unique_count
#CO: what is the purpose of this? Do we want to merge this information about counties back into the main dataframe? If no, then what is the purpose? If yes, then where is that being done?

,source,county_level,article_id_count
0,Atlanta Business Chronicle,Bibb County,3
1,Atlanta Business Chronicle,Butts County,2
2,Atlanta Business Chronicle,Calhoun County,2
3,Atlanta Business Chronicle,Carroll County,1
4,Atlanta Business Chronicle,Cobb County,5
...,...,...,...
787,WRDW News CBS 12 NBC 26,Whitfield County,1
788,WRDW News CBS 12 NBC 26,Wilcox County,10
789,WRDW News CBS 12 NBC 26,Wilkes County,6
790,WRDW News CBS 12 NBC 26,Wilkinson County,23


In [128]:
#df_ng_all_m20_county_unique_count.to_excel('Georgia_m20_county_unique_count_v2.xlsx')
pd.reset_option('display.max_colwidth')
df #CO: in "place ng1", why is the first row empty? America and Georgia are both unigrams, so why woouldn't these be mentioned here?


,index,source,en_source,website,article_url,article_title,article_author,date_published,article_final,ner_places,ner_orgs,article_lemma_num,id,ner_places_orgs,ner_person,place_ng1,place_ng2
0,0,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_167b17...,American Pickers to film in Georgia,NaN,2022-09-19,The American Pickers are excited to return to ...,"{'America (GPE)', 'Georgia (GPE)'}","{'American Pickers (ORG)', 'CDC (ORG)', 'The A...",american picker excited return georgia plan fi...,1,"{American Pickers (ORG), CDC (ORG), Georgia (G...",{Pickers (PERSON)},{},[]
1,1,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_248646...,Free Covid test kits available TODAY,NaN,2022-09-15,Candler County Health Department and Southeast...,{'Metter (GPE)'},"{'SE Broad Street and (ORG)', 'Southeast Healt...",candler county health department southeast hea...,2,"{Candler County Health Department (ORG), N. Ro...",{Covid (PERSON)},{},[]
2,2,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_2eccfd...,CLASS OF 2000 STILL LEADS THE WAY! Over $29K r...,NaN,2022-10-14,After the third week of the Education Foundati...,set(),"{'CarterFranklin (ORG)', 'Ronnie & Janie Sikes...",week education foundation 2022 campaign class ...,3,"{Paul & Beverly MacGregor (ORG), the Education...","{Roberta Graham (PERSON), Jill Vining (PERSON)...","{Allen, Charles, Franklin, Murphy, Rebecca, Ka...",[]
3,3,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_2f555e...,"MMS Lady Tigers to play for championship, MMS ...",NaN,2022-09-27,Congratulations to the Metter Middle School La...,"{'South Tattnall (GPE)', 'Bryan County (GPE)',...","{'MHS (ORG)', 'the Metter Middle School Lady T...",congratulation metter middle school lady tiger...,4,"{Toombs County (GPE), Bryan County (GPE), MHS ...",{South Tattnall (PERSON)},{Metter},[]
4,4,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_2fe7f1...,ATTN Hunters & Anglers: Celebrate Hunting & Fi...,NaN,2022-09-22,In a proclamation recently signed by Gov. Bria...,{'Georgia (GPE)'},"{'Outdoor Adventure Days (ORG)', 'Congressiona...",proclamation recently sign gov brian kemp satu...,5,"{Outdoor Adventure Days (ORG), NHF Day (GPE), ...","{Brian Kemp (PERSON), Richard Nixon (PERSON)}",{},[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,WRDW News CBS 12 NBC 26,1.0,http://www.wrdw.com/,https://www.wrdw.com/video/2022/09/20/watch-ob...,WATCH: Objects appear to be falling from the s...,NaN,2022-09-20,Video taken 9/20/22 6:45 am. Objects were visi...,"{'Calhan (GPE)', 'Hartsel (GPE)', 'Colorado (G...",{'11 News Viewers (ORG)'},video take 92022 645 object visible hartsel ca...,4996,"{Colorado (GPE), Hartsel (ORG), Calhan (GPE)}",{},{},[]
4996,4996,WRDW News CBS 12 NBC 26,1.0,http://www.wrdw.com/,https://www.wrdw.com/video/2022/09/20/watch-uk...,WATCH | UK Healthcare offering ‘Stop the Bleed...,NaN,2022-09-19,UK Healthcare offering ‘Stop the Bleed’ training,set(),{'UK Healthcare (ORG)'},uk healthcare offering stop bleed training,4997,{UK Healthcare (ORG)},{},{},[]
4997,4997,WRDW News CBS 12 NBC 26,1.0,http://www.wrdw.com/,https://www.wrdw.com/video/2022/09/20/waterloo...,Waterloo man arrested after allegedly stealing...,NaN,2022-09-20,Police arrested a Waterloo man who they say st...,{'Lost Island (LOC)'},set(),police arrest waterloo man steal 2500 worth e...,4998,{Lost Island (LOC)},{Waterloo (PERSON)},{},[]
4998,4998,WRDW News CBS 12 NBC 26,1.0,http://www.wrdw.com/,https://www.wrdw.com/video/2022/09/20/western-...,Western Alaska communities still uncovering da...,NaN,2022-09-20,Western Alaska communities still uncovering da...,{'Alaska (GPE)'},set(),western alaska community uncover damage major ...,4999,{Alaska (GPE)},{},{},[]
